0. Scan glass slides at 20x
1. Create Excel sheet with patient information
2. Screen H&E images to select potentially healthy sections
3. Annotate the Region of Interest (ROI) (Optional)
4. Convert ROI XML to Image (Code: roixml2png.py)
5. Convert H&E images to 1um resolution tif to apply DeepLab (Code: rescale_wsi.py)
6. Annotate 12 tissue compartments, apply DeepLab, and evaluate performance metrics
7. Apply HoVerNet
8. Rotate and crop tissue pieces, DLmask, HoVerNet segmented nuclei mask (Code: align_he_dl_cnt.py)
9. Calculate tissue composition (Code: DLcomposition.py)
10. Calculate epidermis thickness, waviness, roughness
12. Calculate dermis thickness, waviness
13. Calculate ECM anisotropy,Interstitial orientation, Thick gap ratio, Thick gap average
14. Calculate tissue morphology
15. ECM cell count and morphology + std using Manual Threshold method
16. Distance from ECM cell to tissue
17. All celltype morphology, orientation, and distance to tissue using HoVerNet
18. Further add features such as more detailed cell subtyping
19.

In [1]:
import os
import glob
import pandas as pd
import numpy as np
from natsort import natsorted

#utilities
from roixml2png import roixml2png
from rescale_wsi import rescale_wsi
from align_he_dl_cnt import align_he_dl_cnt

In [2]:
NDPI = r'\\fatherserverdw\kyuex\clue images'
oneum = r'\\fatherserverdw\kyuex\clue images\1um'
imcrop = r'\\fatherserverdw\kyuex\clue images\1um\imcrop'
imcroproi = r'\\fatherserverdw\kyuex\clue images\1um\imcrop_roi'

dlmask = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined'
dlcrop = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined\dlcrop'
dlcroproi = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined\dlcrop_roi'

roixml = r'\\fatherserverdw\kyuex\clue images\annotations\roi'
roimask_small = r'\\fatherserverdw\kyuex\clue images\annotations\roi\labeledmask_20rsf'
tissue_area_small = r'\\fatherserverdw\kyuex\clue images\annotations\roi\TA_20rsf'

twelve_annot = r'\\fatherserverdw\kyuex\clue images\annotations\12class'

LUT = r"\\fatherserverdw\kyuex\imlist_all.xlsx"

In [3]:
#Load healthy tissue look up table (LUT)
LUT = pd.read_excel(LUT)
# xmlist = LUT['filename'][(LUT['student score']>1) & (LUT['Block or Slide?']=="Both")]
LUThealthy = LUT[(LUT['student score']>1)]
LUThealthyback = LUThealthy[LUThealthy['body part 1'].str.lower()=="back"]

In [5]:
#Step 4: ROI XML 2 Image
#Saves Image to where XML is in subfolders

xmlist = [os.path.join(roixml,xmlpth.replace('ndpi','xml')) for xmlpth in LUThealthyback['filename']]
[roixml2png(xmlpth,NDPI) for xmlpth in xmlist]
# pd.DataFrame(np.array(roiinfo),columns=['fn','ROIA','TA','ratio']).to_csv(r"\\fatherserverdw\kyuex\ROITA_ratio.csv")

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [6]:
#Step 5: rescale wsi to 1um for applying deeplab
#Saves Image to where wsi is in subfolder
wsis = [os.path.join(NDPI,_) for _ in LUThealthy.filename]
wsis = natsorted(wsis)
for idx,wsi in enumerate(wsis):
    rescale_wsi(wsi,1)
    print(idx,'/',len(wsis))

0 / 216
1 / 216
2 / 216
3 / 216
4 / 216
5 / 216
6 / 216
7 / 216
8 / 216
9 / 216
10 / 216
11 / 216
12 / 216
13 / 216
14 / 216
15 / 216
16 / 216
17 / 216
18 / 216
19 / 216
20 / 216
21 / 216
22 / 216
23 / 216
24 / 216
25 / 216
26 / 216
27 / 216
28 / 216
29 / 216
30 / 216
31 / 216
32 / 216
33 / 216
34 / 216
35 / 216
36 / 216
37 / 216
38 / 216
39 / 216
40 / 216
41 / 216
42 / 216
43 / 216
44 / 216
45 / 216
46 / 216
47 / 216
48 / 216
49 / 216
50 / 216
51 / 216
52 / 216
53 / 216
54 / 216
55 / 216
56 / 216
57 / 216
58 / 216
59 / 216
60 / 216
61 / 216
62 / 216
63 / 216
64 / 216
65 / 216
66 / 216
67 / 216
68 / 216
69 / 216
70 / 216
71 / 216
72 / 216
73 / 216
74 / 216
75 / 216
76 / 216
77 / 216
78 / 216
79 / 216
80 / 216
81 / 216
82 / 216
83 / 216
84 / 216
85 / 216
86 / 216
87 / 216
88 / 216
89 / 216
90 / 216
91 / 216
92 / 216
93 / 216
94 / 216
95 / 216
96 / 216
97 / 216
98 / 216
99 / 216
100 / 216
101 / 216
102 / 216
103 / 216
104 / 216
105 / 216
106 / 216
107 / 216
108 / 216
109 / 216
110 / 216


In [3]:
#8. Rotate and crop tissue pieces and DLmask (Code)
#9. Rotate HoVerNet segmented nuclei mask (Code)

# LUTpth = r"\\fatherserverdw\kyuex\datadst\WSI_rotation_crop_LUT.xlsx"
# ClUedegrot_df = pd.read_excel(LUTpth).astype("object")

In [4]:
cntsrc = r'\\fatherserverdw\kyuex\clue images\hovernet_out\json'
dlsrc = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined'
roisrc = r'\\fatherserverdw\kyuex\clue images\annotations\roi\labeledmask_20rsf'
def hasjson(row):
    fn,ext = os.path.splitext(row)
    return os.path.exists(os.path.join(cntsrc,fn+'.json'))
def hasdl(row):
    fn,ext = os.path.splitext(row)
    return os.path.exists(os.path.join(dlsrc,fn+'.tif'))
def hasroi(row):
    fn,ext = os.path.splitext(row)
    return os.path.exists(os.path.join(roisrc,fn+'.png'))
LUThealthyback['hasjson']=LUThealthyback['filename'].apply(lambda row: hasjson(row))
LUThealthyback['hasdl']=LUThealthyback['filename'].apply(lambda row: hasdl(row))
LUThealthyback['hasroi']=LUThealthyback['filename'].apply(lambda row: hasroi(row))

C:\Users\labuser\AppData\Local\Temp\ipykernel_1964\2748685258.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LUThealthyback['hasjson']=LUThealthyback['filename'].apply(lambda row: hasjson(row))


In [9]:
LUThealthyback2 = LUThealthyback[(LUThealthyback['hasjson']==True) & (LUThealthyback['hasdl']==True) & (LUThealthyback['hasroi']==True)]
LUThealthyback2 = LUThealthyback2.reset_index()
len(LUThealthyback2),len(LUThealthyback[(LUThealthyback['hasjson']==True)]),len(LUThealthyback[(LUThealthyback['hasdl']==True)]),len(LUThealthyback)

(53, 53, 66, 66)

In [15]:
cntsrc = r'\\fatherserverdw\kyuex\clue images\hovernet_out\json'
roisrc = r'\\fatherserverdw\kyuex\clue images\annotations\roi\labeledmask_20rsf'
wsisrc = r'\\fatherserverdw\kyuex\clue images'
dlsrc = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined'
dst = r'\\fatherserverdw\kyuex\datadst\20220929'

for idx,row in LUThealthyback2.iterrows():
    print(idx,'/',len(LUThealthyback2))
    fn,ext = os.path.splitext(row['filename'])
    try:
        rotationdf = align_he_dl_cnt(dst,fn,wsisrc,dlsrc,cntsrc,roisrc)
    except:
        print(fn)

0 / 53
section N:  1 / 1
2022-06-07 14.27.44
1 / 53
section N:  1 / 1
2022-06-07 14.32.24
2 / 53
section N:  1 / 1
3 / 53
section N:  1 / 1
4 / 53
section N:  1 / 1
5 / 53
6 / 53
7 / 53
8 / 53
9 / 53
section N:  1 / 2
section N:  2 / 2
10 / 53
section N:  1 / 3
section N:  2 / 3
section N:  3 / 3
11 / 53
12 / 53
13 / 53
14 / 53
15 / 53
16 / 53
17 / 53
18 / 53
19 / 53
20 / 53
section N:  1 / 1
2022-06-13 11.45.50
21 / 53
22 / 53
section N:  1 / 1
2022-06-13 11.51.23
23 / 53
24 / 53
25 / 53
section N:  1 / 1
2022-06-13 12.01.11
26 / 53
27 / 53
section N:  1 / 1
28 / 53
29 / 53
section N:  1 / 1
30 / 53
section N:  1 / 4
section N:  2 / 4
section N:  3 / 4
section N:  4 / 4
31 / 53
32 / 53
section N:  1 / 4
section N:  2 / 4
section N:  3 / 4
section N:  4 / 4
33 / 53
section N:  1 / 1
34 / 53
section N:  1 / 3
section N:  2 / 3
section N:  3 / 3
35 / 53
section N:  1 / 1
36 / 53
37 / 53
38 / 53
39 / 53
40 / 53
41 / 53
42 / 53
43 / 53
44 / 53
45 / 53
46 / 53
47 / 53
section N:  1 / 1
48 /

In [17]:
imcropsrc = r'\\fatherserverdw\kyuex\datadst\20220929\imcrop'
def hasimcrop(row):
    fn,ext = os.path.splitext(row)
    fn = fn+'sec1'
    return os.path.exists(os.path.join(imcropsrc,fn+'.png'))
LUThealthyback2['hasimcrop']=LUThealthyback2['filename'].apply(lambda row: hasimcrop(row))
LUThealthyback3=LUThealthyback2[LUThealthyback2['hasimcrop']]
LUThealthyback3

,level_0,index,gender,age,pid,path,filename,year,sample id,label name,...,student score,ROI ANNOTATED (0 :not annotated 1.annotated),roi reviewed,Nroi,Inbal Score,Inbal comment,hasjson,hasdl,hasroi,hasimcrop
2,54,1143,F,85.604454,471,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-07 14.34.59.ndpi,10,3702,10-3702,...,5,1,1.0,NaN,NaN,NaN,True,True,True,True
3,55,1144,F,85.604454,471,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-07 14.37.19.ndpi,10,3702,10-3702,...,3,1,1.0,NaN,NaN,NaN,True,True,True,True
4,105,1173,M,92.435616,484,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-07 17.24.15.ndpi,10,6634,10-6634,...,2,1,1.0,NaN,NaN,NaN,True,True,True,True
5,110,162,M,74.745205,71,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-07 17.43.53.ndpi,3,18469,03-18469,...,4,1,1.0,NaN,4.0,can use the third and fourth (marked) sections,True,True,True,True
6,119,394,M,87.602740,178,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-08 17.04.20.ndpi,5,9867,05:9867,...,2,1,1.0,NaN,NaN,NaN,True,True,True,True
7,121,386,M,83.120548,177,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-08 17.14.34.ndpi,5,9840,05:9840,...,2,1,1.0,NaN,2.0,can use the piece on the right (but would avoi...,True,True,True,True
8,122,388,M,83.120548,177,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-08 17.25.47.ndpi,5,9840,05:9840,...,2,1,1.0,NaN,NaN,NaN,True,True,True,True
9,160,433,M,79.104110,191,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-08 18.08.15.ndpi,5,12047,05:12047,...,4,1,1.0,NaN,4.0,"1,2,4th piece are all okay (third piece has lo...",True,True,True,True
10,161,434,M,79.104110,191,\\fatherserverdw\Q\research\images\clue_batch2...,2022-06-08 18.13.05.ndpi,5,12047,05:12047,...,4,1,1.0,NaN,NaN,NaN,True,True,True,True
11,174,324,F,67.542466,149,\\fatherserverdw\kyuex\clue images\2022-06-09 ...,2022-06-09 11.46.15.ndpi,5,2509,05:2509,...,2,1,1.0,NaN,NaN,NaN,True,True,True,True


In [ ]:
f